In [ ]:
import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, UpSampling2D, Concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from collections import defaultdict
import os
from tensorflow.keras.utils import Sequence

In [ ]:
DATA_ROOT = Path("C:/Users/2955352g/Desktop/pig_data_edinburgh")

In [ ]:
class AnnotatedDataGenerator(Sequence):
    """Data generator for annotated frames to avoid memory issues"""
    
    def __init__(self, data_info, batch_size=8, target_size=(224, 224), shuffle=True):
        self.data_info = data_info
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.indices = np.arange(len(data_info))
        if shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.data_info) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_data = [self.data_info.iloc[i] for i in batch_indices]
        
        X, y = self._load_batch(batch_data)
        return X, y
    
    def _load_batch(self, batch_data):
        X = []
        y = []
        
        for data in batch_data:
            try:
                # Load frame from video
                cap = cv2.VideoCapture(str(data['video_path']))
                cap.set(cv2.CAP_PROP_POS_FRAMES, data['frame_num'])
                ret, frame = cap.read()
                cap.release()
                
                if ret:
                    # Resize frame
                    frame = cv2.resize(frame, self.target_size)
                    frame = frame.astype(np.float32) / 255.0  # Normalize
                    X.append(frame)
                    y.append(data['behavior_encoded'])
                    
            except Exception as e:
                print(f"Error loading frame: {e}")
                continue
        
        return np.array(X), np.array(y)
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [ ]:
def load_annotated_data_info():
    """Load annotated data information without loading frames into memory"""
    data_info = []
    
    # Check if annotated directory exists
    annotated_dir = DATA_ROOT / "annotated"
    if not annotated_dir.exists():
        print(f"Warning: Annotated directory {annotated_dir} does not exist")
        return pd.DataFrame()
    
    json_files = list(annotated_dir.rglob("output.json"))
    if not json_files:
        print("No output.json files found in annotated directory")
        return pd.DataFrame()
    
    print(f"Found {len(json_files)} annotation files")
    
    for json_file in json_files:
        try:
            with open(json_file) as f:
                data = json.load(f)
            
            video_path = json_file.parent / "color.mp4"
            if not video_path.exists():
                print(f"Warning: Video file {video_path} does not exist")
                continue
            
            # Quick check if video can be opened
            cap = cv2.VideoCapture(str(video_path))
            if not cap.isOpened():
                print(f"Warning: Cannot open video {video_path}")
                continue
            
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            
            print(f"Processing {video_path} with {total_frames} frames")
            
            for obj in data.get('objects', []):
                for frame_data in obj.get('frames', []):
                    frame_num = frame_data.get('frameNumber', 0)
                    if frame_num >= total_frames:
                        continue
                    
                    # Handle different bbox formats
                    bbox = frame_data.get('bbox', [0, 0, 100, 100])
                    
                    # Debug: Print first few bbox formats
                    if len(data_info) < 3:
                        print(f"  Sample bbox: {bbox}, type: {type(bbox)}")
                    
                    # Normalize bbox format
                    try:
                        if isinstance(bbox, str):
                            import ast
                            bbox = ast.literal_eval(bbox)
                        
                        if isinstance(bbox, dict):
                            # Convert dict format to list [x, y, width, height]
                            x = bbox.get('x', 0)
                            y = bbox.get('y', 0)
                            w = bbox.get('width', bbox.get('w', 100))
                            h = bbox.get('height', bbox.get('h', 100))
                            bbox = [x, y, w, h]
                        
                        # Ensure bbox is a list with 4 elements
                        if not isinstance(bbox, (list, tuple)) or len(bbox) != 4:
                            bbox = [0, 0, 100, 100]
                    
                    except Exception as e:
                        print(f"Error processing bbox {bbox}: {e}")
                        bbox = [0, 0, 100, 100]
                    
                    data_info.append({
                        'video_path': str(video_path),
                        'behavior': frame_data.get('behaviour', 'unknown'),
                        'bbox': bbox,
                        'video_id': json_file.parent.name,
                        'date': json_file.parent.parent.name,
                        'frame_num': frame_num
                    })
                    
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            continue
    
    print(f"Found {len(data_info)} annotated frames")
    return pd.DataFrame(data_info)


In [ ]:
def load_sample_frames(data_info, sample_size=100):
    """Load a small sample of frames for quick testing"""
    if len(data_info) > sample_size:
        sample_info = data_info.sample(n=sample_size).reset_index(drop=True)
    else:
        sample_info = data_info.copy()
    
    frames = []
    valid_indices = []
    
    for idx, row in sample_info.iterrows():
        try:
            cap = cv2.VideoCapture(str(row['video_path']))
            cap.set(cv2.CAP_PROP_POS_FRAMES, row['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if ret:
                frame = cv2.resize(frame, (224, 224))
                frame = frame.astype(np.float32) / 255.0
                frames.append(frame)
                valid_indices.append(idx)
                
        except Exception as e:
            print(f"Error loading frame {idx}: {e}")
            continue
    
    return np.array(frames), sample_info.iloc[valid_indices]

In [ ]:
def build_simple_detector(input_shape=(224, 224, 3)):
    """Simplified detector for memory efficiency"""
    inputs = Input(input_shape)
    
    # Simple CNN for detection
    x = Conv2D(32, (3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = Conv2D(64, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    x = Conv2D(128, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    x = Conv2D(256, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    
    # Global average pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    # Output layer for bounding box regression
    output = Dense(4, activation='sigmoid')(x)  # [x, y, w, h] normalized
    
    return Model(inputs, output)

In [ ]:
def train_detector(data_info):
    """Train detector using data generator"""
    if data_info.empty:
        print("No annotated data available for training detector")
        return None
    
    print(f"Training detector on {len(data_info)} samples")
    
    # Debug: Check bbox format
    print("Sample bbox data:")
    for i, row in data_info.head(3).iterrows():
        print(f"  Row {i}: bbox = {row['bbox']}, type = {type(row['bbox'])}")
    
    # Use only a sample for detector training to save memory
    sample_size = min(1000, len(data_info))
    sample_frames, sample_info = load_sample_frames(data_info, sample_size)
    
    if len(sample_frames) == 0:
        print("No frames could be loaded for detector training")
        return None
    
    # Prepare normalized bounding boxes
    y_train = []
    for _, row in sample_info.iterrows():
        bbox = row['bbox']
        
        # Handle different bbox formats
        try:
            if isinstance(bbox, str):
                # If bbox is a string, try to parse it
                import ast
                bbox = ast.literal_eval(bbox)
            
            if isinstance(bbox, dict):
                # If bbox is a dictionary, extract values
                x = bbox.get('x', 0)
                y = bbox.get('y', 0)
                w = bbox.get('width', bbox.get('w', 100))
                h = bbox.get('height', bbox.get('h', 100))
                bbox = [x, y, w, h]
            
            # Ensure bbox is a list/array with 4 elements
            if not isinstance(bbox, (list, tuple, np.ndarray)) or len(bbox) != 4:
                print(f"Invalid bbox format: {bbox}, using default")
                bbox = [0, 0, 100, 100]
            
            # Normalize bbox (assuming original frame size is 1280x720)
            x_norm = float(bbox[0]) / 1280.0
            y_norm = float(bbox[1]) / 720.0
            w_norm = float(bbox[2]) / 1280.0
            h_norm = float(bbox[3]) / 720.0
            
            # Clamp values to [0, 1]
            x_norm = max(0, min(1, x_norm))
            y_norm = max(0, min(1, y_norm))
            w_norm = max(0, min(1, w_norm))
            h_norm = max(0, min(1, h_norm))
            
            y_train.append([x_norm, y_norm, w_norm, h_norm])
            
        except Exception as e:
            print(f"Error processing bbox {bbox}: {e}")
            # Use default normalized bbox
            y_train.append([0.1, 0.1, 0.2, 0.2])
    
    y_train = np.array(y_train)
    
    # Build and compile model
    model = build_simple_detector()
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    print("Training detector...")
    history = model.fit(
        sample_frames, y_train,
        epochs=10,
        batch_size=16,
        validation_split=0.2,
        verbose=1
    )
    
    return model

In [ ]:
def build_behavior_classifier(input_shape=(224,224,3), num_classes=5):
    """Lightweight behavior classifier"""
    inputs = Input(input_shape)
    
    # Use a smaller CNN instead of ResNet50 for memory efficiency
    x = Conv2D(32, (3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = Conv2D(64, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    x = Conv2D(128, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs, predictions)

In [ ]:
def train_behavior_classifier(data_info):
    """Train behavior classifier using data generator"""
    if data_info.empty:
        print("No annotated data available for training behavior classifier")
        return None
    
    print(f"Training behavior classifier on {len(data_info)} samples")
    
    # Encode behaviors
    unique_behaviors = data_info['behavior'].unique()
    behavior_to_idx = {behavior: idx for idx, behavior in enumerate(unique_behaviors)}
    
    data_info_copy = data_info.copy()
    data_info_copy['behavior_encoded'] = data_info_copy['behavior'].map(behavior_to_idx)
    
    # Convert to one-hot encoding
    num_classes = len(unique_behaviors)
    encoded_behaviors = []
    for idx in data_info_copy['behavior_encoded']:
        one_hot = np.zeros(num_classes)
        one_hot[idx] = 1
        encoded_behaviors.append(one_hot)
    
    data_info_copy['behavior_encoded'] = encoded_behaviors
    
    print(f"Found {num_classes} behavior classes: {list(unique_behaviors)}")
    
    # Build model
    model = build_behavior_classifier(num_classes=num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Create data generator
    train_generator = AnnotatedDataGenerator(
        data_info_copy, 
        batch_size=16, 
        target_size=(224, 224)
    )
    
    print("Training behavior classifier...")
    history = model.fit(
        train_generator,
        epochs=5,
        verbose=1
    )
    
    return model, behavior_to_idx

In [ ]:
class PigTracker:
    def __init__(self, max_age=8):
        self.trackers = defaultdict(dict)
        self.max_age = max_age
        self.next_id = 0
    
    def update(self, frame, detections):
        """Update trackers with new detections"""
        active_trackers = {}
        
        # Update existing trackers
        for tid in list(self.trackers.keys()):
            tracker = self.trackers[tid]['tracker']
            bbox = self.trackers[tid]['bbox']
            
            # Update tracker with new frame
            success, new_bbox = tracker.update(frame)
            
            if success:
                active_trackers[tid] = {
                    'bbox': new_bbox,
                    'tracker': tracker,
                    'age': 0
                }
            elif self.trackers[tid]['age'] < self.max_age:
                active_trackers[tid] = {
                    'bbox': bbox,
                    'tracker': tracker,
                    'age': self.trackers[tid]['age'] + 1
                }
        
        # Create new trackers for unmatched detections
        for det in detections:
            matched = False
            for tid in active_trackers:
                if self._iou(det['bbox'], active_trackers[tid]['bbox']) > 0.4:
                    matched = True
                    break
            
            if not matched:
                # Use CSRT tracker
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, tuple(det['bbox']))
                active_trackers[self.next_id] = {
                    'bbox': det['bbox'],
                    'tracker': tracker,
                    'age': 0
                }
                self.next_id += 1
        
        self.trackers = active_trackers
        return self.trackers

In [ ]:
    def _iou(self, box1, box2):
        """Calculate Intersection over Union"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[0]+box1[2], box2[0]+box2[2])
        y2 = min(box1[1]+box1[3], box2[1]+box2[3])
        
        inter_area = max(0, x2 - x1) * max(0, y2 - y1)
        box1_area = box1[2] * box1[3]
        box2_area = box2[2] * box2[3]
        
        return inter_area / float(box1_area + box2_area - inter_area)


In [ ]:
def run_full_pipeline():
    print("Starting pig behavior detection pipeline...")
    
    # Check if data directory exists
    if not DATA_ROOT.exists():
        print(f"Error: Data directory {DATA_ROOT} does not exist")
        return None, None, None
    
    print("Loading data information...")
    data_info = load_annotated_data_info()
    
    if data_info.empty:
        print("No annotated data found. Cannot proceed with training.")
        return None, None, None
    
    print("Training detector...")
    detector = train_detector(data_info)
    
    print("Initializing tracker...")
    tracker = PigTracker()
    
    print("Training behavior classifier...")
    behavior_model, behavior_mapping = train_behavior_classifier(data_info)
    
    print("Pipeline training complete!")
    return detector, tracker, behavior_model

In [ ]:
def test_single_video(detector, behavior_model, video_path):
    """Test the trained models on a single video"""
    if not os.path.exists(video_path):
        print(f"Video file {video_path} does not exist")
        return
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video {video_path}")
        return
    
    print(f"Processing video: {video_path}")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:  # Process every 30th frame
            # Resize frame for models
            frame_resized = cv2.resize(frame, (224, 224))
            frame_norm = frame_resized.astype(np.float32) / 255.0
            frame_batch = np.expand_dims(frame_norm, axis=0)
            
            # Get predictions
            if detector:
                bbox_pred = detector.predict(frame_batch, verbose=0)
                print(f"Frame {frame_count}: Detected bbox: {bbox_pred[0]}")
            
            if behavior_model:
                behavior_pred = behavior_model.predict(frame_batch, verbose=0)
                behavior_class = np.argmax(behavior_pred[0])
                confidence = np.max(behavior_pred[0])
                print(f"Frame {frame_count}: Behavior class: {behavior_class}, Confidence: {confidence:.3f}")
    
    cap.release()
    print(f"Processed {frame_count} frames")

In [1]:
if __name__ == "__main__":
    # Set memory growth for GPU if available
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)
    
    detector, tracker, behavior_model = run_full_pipeline()
    
    if detector is not None:
        print("Detection model trained successfully")
    if behavior_model is not None:
        print("Behavior classification model trained successfully")
    if tracker is not None:
        print("Tracker initialized successfully")
    
    # Test on a single video if models are trained
    if detector or behavior_model:
        test_video_path = DATA_ROOT / "annotated" / "test_video.mp4"  # Adjust path as needed
        if test_video_path.exists():
            test_single_video(detector, behavior_model, str(test_video_path))

Starting pig behavior detection pipeline...
Loading data information...
Found 12 annotation files
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_05\000002\color.mp4 with 1800 frames
  Sample bbox: {'x': 196, 'y': 299, 'width': 240, 'height': 121}, type: <class 'dict'>
  Sample bbox: {'x': 196, 'y': 290, 'width': 261, 'height': 122}, type: <class 'dict'>
  Sample bbox: {'x': 196, 'y': 283, 'width': 251, 'height': 190}, type: <class 'dict'>
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_05\000009\color.mp4 with 1800 frames
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_11\000016\color.mp4 with 1800 frames
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_11\000028\color.mp4 with 1800 frames
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_11\000036\color.mp4 with 1800 frames
Processing C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\2019_11_15\000033\co

C:\Users\2955352g\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
791/791 ━━━━━━━━━━━━━━━━━━━━ 6106s 8s/step - accuracy: 0.3316 - loss: 2.0345
Epoch 2/5
791/791 ━━━━━━━━━━━━━━━━━━━━ 6137s 8s/step - accuracy: 0.3421 - loss: 1.8600
Epoch 3/5
791/791 ━━━━━━━━━━━━━━━━━━━━ 15713s 20s/step - accuracy: 0.3497 - loss: 1.7574
Epoch 4/5
791/791 ━━━━━━━━━━━━━━━━━━━━ 6008s 8s/step - accuracy: 0.3540 - loss: 1.7346
Epoch 5/5
791/791 ━━━━━━━━━━━━━━━━━━━━ 5771s 7s/step - accuracy: 0.3664 - loss: 1.6904
Pipeline training complete!
Detection model trained successfully
Behavior classification model trained successfully
Tracker initialized successfully
Processing video: C:\Users\2955352g\Desktop\pig_data_edinburgh\annotated\test_video.mp4
Frame 30: Detected bbox: [0.4492801  0.4113723  0.16241232 0.27503106]
Frame 30: Behavior class: 0, Confidence: 0.341
Frame 60: Detected bbox: [0.44819766 0.40995285 0.15757634 0.27056196]
Frame 60: Behavior class: 0, Confidence: 0.332
Frame 90: Detected bbox: [0.44944987 0.41172624 0.16300143 0.2757967 ]
Frame 90: Behavior 

usage: ipykernel_launcher.py [-h] [--device DEVICE]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\2955352g\AppData\Roaming\jupyter\runtime\kernel-v39555ef68165e426e0c65bde4934e1faca2dffe1e.json


SystemExit: 2

c:\Users\2955352g\.conda\envs\tf_env\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
